## Iris klassifikasjon

Iris datasettet består av 150 datapunkter som beskriver blomster. Det inneholder tre forskjellige spesies av iris blomsterfamilien.

In [ ]:
# imports
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# lese inn data
iris = load_iris()
iris_labels = iris.target_names[iris['target']]
X, y = iris.data, iris.target

Her deler vi data i 60% treningsdata, 20% valideringsdata og 20% testdata. 

In [ ]:
# dele data i trenings, validerings og testdata
# generer X_train, X_val, X_test, y_train, y_val, y_test

x_train, x_val, y_train, y_val = train_test_split(
    X, y, shuffle=False, test_size=0.4
)

x_val, x_test, y_val, y_test = train_test_split(
    x_val, y_val, shuffle=False, test_size=0.5
)

print(x_test)
print(y_test)


Regn ut de første to prinsipale komponenter PC1 og PC2 på treningsdata. Lag et dataframe `pca_df` med to kolonner `PC1` og `PC2` som inneholder de transformerte data. Så lag en scatterfigur med PC1 på x-aksen, PC2 på y-aksen, fargelagt etter type irisplante. Tips: Bruk [`sklearn.decomposition.PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). Du har allerede gjort dette i visualiserings-aktivitetet. 

In [ ]:
pca = PCA(n_components=2)
pca.fit(x_train)

pca_df = pd.DataFrame(pca.transform(x_train), 
                      columns=['PC1', 'PC2'])
pca_df["iris_labels"] = iris.target_names[y_train]

print(pca_df)
# pca_df["iris_labels"] = iris.target_names[iris['target']]
# Scatterplot
fig = px.scatter(data_frame=pca_df, 
                 x='PC1', 
                 y='PC2', 
                 color=pca_df["iris_labels"],
                 labels={'color':'Type'})
fig.show()

Bruk en statistisk test for å se om det er forskjeller i PC1 og PC2 mellom de ulike klassene. 

In [ ]:
for pca_dim in [0,1]:
    for class1 in range(3):
        for class2 in range(class1):

            data1 = pca_df[pca_df['iris_labels'] == iris.target_names[class1]][pca_dim]
            data2 = pca_df[pca_df['iris_labels'] == iris.target_names[class2]][pca_dim]

            t_stat, p_val = stats.ttest_ind(
                data1, data2
                )
            pval = stats.ttest_ind()
            print(pval)

Lag en k-nærmeste nabo klassifikasjonsmodell for k=1, k=5, k=10, k=20, k=30 på de prinsipale komponentene. Tegn en contour-figur for å se hva som blir klassifisert som hvilken klasse. Hva skjer når k blir større? Tips: Et eksempel er vist under. 

In [ ]:
# tilpass 1-NN klassifikasjonsmodell
output_dict = {}

for neigh in [1,5,10,20,30]:
    clf = KNeighborsClassifier(n_neighbors=neigh)
    clf.fit(pca.transform(x_train), y_train)

    # lag en grid 
    grid_margins = (np.arange(-4, 4.01, 0.02), 
                    np.arange(-1.5, 1.51, 0.02))
    lens = [len(margin) for margin in grid_margins]
    lens.reverse()
    grid = np.stack(np.meshgrid(*grid_margins), -1).reshape(-1, len(grid_margins))

    # klassifiser på grid
    grid_predictions = clf.predict(grid)

    output_dict[neigh] = {"clf":clf, "clf_grid":grid_predictions}

    decision_boundaries = grid_predictions.reshape(lens)

        # Scatterplott av data
    fig = px.scatter(data_frame=pca_df, 
                    x='PC1', 
                    y='PC2', 
                    color=y_train, 
                    color_continuous_scale= [(0, '#0173b2'), (0.33, '#0173b2'),
                                            (0.33, '#029e73'), (0.67, '#029e73'),
                                            (0.67, '#de8f05'), (1, '#de8f05')])
    # legg til contourplott
    fig.add_trace(go.Contour(
        z=decision_boundaries,
        x=grid_margins[0],
        y=grid_margins[1], 
        colorscale = [(0, '#b2d4e7'), (0.5, '#b4e1d4'), (1, '#f2ceb2')],
        line_smoothing=0,
        showscale=False))

    # endre layout og vis figuren
    fig.update_layout(template="plotly_white", 
                    coloraxis_colorbar=dict(
                        title="Species",
                        tickvals=[0.3,1,1.7],
                        ticktext=iris.target_names,
                        lenmode="pixels", 
                        len=200))
    fig.show()


In [ ]:
classifiers = {n_neighbors: ___ for n_neighbors in [1, 5, 10, 20, 30]}
for key, clf in classifiers.items():
    ___

Regn ut nøyaktighet av alle modellene dine på valideringsdata. Du må bruke PCA som er trent på treningsdata og bruke transform-metoden på valideringsdata. 

In [44]:
# nøyaktighet på valideringsdata

pca = PCA(n_components=2)
pca.fit(x_train)

validation_accuracies = {}
for k, v in output_dict.items():
    knn_clf = v["clf"]
    x_val_transformed = pca.transform(x_val)
    y_val_pred = knn_clf.predict(x_val_transformed)
    accuracy = accuracy_score(y_val, y_val_pred)
    validation_accuracies[k] = accuracy

print(validation_accuracies)

{1: 0.3333333333333333, 5: 0.3333333333333333, 10: 0.3333333333333333, 20: 0.3333333333333333, 30: 0.3333333333333333}


Regn ut gerneraliseringsevnen av modellen vår på testdata. 

In [ ]:
# generaliseringsevnen
best_model = KNeighborsClassifier(n_neighbors=10)
best_model.fit(x_train, y_train) 
generalisation_acc = accuracy_score(y_test, best_model.predict(pca.transform(x_test)) )